In [ ]:
import os
# 

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.llms import HuggingFacePipeline
# from langchain_core.pydantic_v1 import BaseModel, Field
# from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnablePassthroughWithInput
from langchain.llms import HuggingFaceHub, huggingface_endpoint
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
## set token as env variable

In [3]:
llm = HuggingFaceHub(
    repo_id="google/flan-t5-large", 
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31724\3287684676.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
e:\langchain-intro\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = "What is the capital of France?"

## Basic Inference Using Huggin Face and Langchain

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, pipeline
import os



# Load locally (make sure you have enough GPU/CPU resources)
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model =AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create a pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Wrap it in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Use in LangChain
prompt = "What is the capital of France?"
response = llm.invoke(prompt)
print(response)


Device set to use cpu


paris


## Prompt Template and LLM Chains

In [13]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine}. Suggest fancy name for this"
)

prompt_template.format(cuisine='Indian')


'I want to open a restaurant for Indian. Suggest fancy name for this'

In [15]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm = llm,
    prompt = prompt_template
)

chain.run(cuisine='Indian')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7940\3993948497.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7940\3993948497.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(cuisine='Indian')


'Indian restaurant'

### Multiple Prompt One after the other

In [21]:
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model =AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create a pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Wrap it in LangChain
llm = HuggingFacePipeline(pipeline=pipe) 

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine}. Suggest fancy name for this"
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest a few menu items for restaurant {restaurant_name}. return them as csv"
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

Device set to use cpu


In [22]:
food_items_chain.run(restaurant_name=name_chain.run(cuisine='Indian'))

'tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tando'

In [23]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run('Indian')
print(response)

tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tandoori chicken tando


#### Get output from both the chains

In [25]:
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model =AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create a pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Wrap it in LangChain
llm = HuggingFacePipeline(pipeline=pipe) 

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine}. Suggest fancy name for this"
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key = 'restaurant_name')

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest a few menu items for restaurant {restaurant_name}. return them as csv"
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key='menu_items')

Device set to use cpu


In [33]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

response = chain({'cuisine':'Mexican'})
print(response)

{'cuisine': 'Mexican', 'restaurant_name': 'chile relleno', 'menu_items': 'chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile relleno chile '}
